In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
messages = [line.rstrip() for line in open('smsspamcollection/SMSSpamCollection')]
print(len(messages))

5574


In [3]:
for message_no, message in enumerate(messages[:10]):
    print(message_no, message)
    print('\n')

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


1 ham	Ok lar... Joking wif u oni...


2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


3 ham	U dun say so early hor... U c already then say...


4 ham	Nah I don't think he goes to usf, he lives around here though


5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv


6 ham	Even my brother is not like to speak with me. They treat me like aids patent.


7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune


8 spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code 

In [4]:
messages = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t',
                           names=["label", "message"])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
messages.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      4825   4516                             Sorry, I'll call later   30
spam      747    653  Please call our customer service representativ...    4

In [6]:
messages['length'] = messages['message'].apply(len)
messages.head()


,label,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [7]:
messages.length.describe()

count    5572.000000
mean       80.489950
std        59.942907
min         2.000000
25%        36.000000
50%        62.000000
75%       122.000000
max       910.000000
Name: length, dtype: float64

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/snehapatra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.auto import tqdm
import time

In [10]:
messages.isna().any()

label      False
message    False
length     False
dtype: bool

In [11]:
messages['message'].nunique()

5169

In [12]:
messages['message'].drop_duplicates(inplace = True)

In [13]:
from collections import Counter
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehapatra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:

all_text = ' '.join(messages['message'].values)

all_text = re.sub(r'http\S+', '', all_text)
all_text = re.sub(r'@\S+', '', all_text)
all_text = re.sub(r'#\S+', '', all_text)

In [15]:
words = all_text.split()

In [16]:

stop_words = set(stopwords.words('english'))
words = [word for word in words if not word in stop_words]

In [17]:

word_counts = Counter(words)
top_words = word_counts.most_common(100)
top_words

[('I', 1469),
 ('u', 698),
 ('2', 457),
 ('call', 376),
 ('get', 326),
 ('U', 300),
 ('ur', 297),
 ("I'm", 286),
 ('&lt;', 276),
 ('You', 263),
 ('4', 255),
 ('.', 241),
 ('go', 235),
 ('like', 224),
 ('know', 224),
 ('got', 204),
 ('come', 199),
 ('?', 187),
 ('...', 163),
 ('want', 158),
 ('Call', 155),
 ('time', 154),
 ('send', 150),
 ('going', 142),
 ('need', 142),
 ('n', 140),
 ("I'll", 138),
 ('How', 137),
 ('still', 135),
 ('If', 133),
 ('one', 133),
 ('But', 131),
 ('No', 126),
 ('text', 126),
 ('Just', 119),
 ('We', 119),
 ('So', 117),
 ('see', 114),
 ('love', 114),
 ('good', 114),
 ('think', 113),
 ('Do', 113),
 ('r', 113),
 ('back', 111),
 ('&', 111),
 ('home', 107),
 ('ü', 105),
 ('tell', 104),
 ('Your', 104),
 ('day', 103),
 ('What', 102),
 ('take', 101),
 ('My', 101),
 ('dont', 100),
 ('free', 99),
 ('And', 98),
 ('Ok', 97),
 ('me.', 97),
 ('The', 96),
 ('A', 96),
 ('mobile', 94),
 ("i'm", 91),
 ('FREE', 90),
 ('make', 90),
 ('new', 89),
 ('give', 89),
 ('-', 88),
 ('phon

In [18]:

def clean(text):
    
    text = re.sub('<.*?>', '', text)
    
    text = re.sub('[^a-zA-Z]', ' ', text).lower()
    
    words = nltk.word_tokenize(text)
   
    words = [w for w in words if w not in stopwords.words('english')]
    
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]
   
    text = ' '.join(words)
    return text

In [19]:
%%time

tqdm.pandas()

messages['cleaned'] = messages['message'].progress_apply(clean)

  0%|          | 0/5572 [00:00<?, ?it/s]

CPU times: user 12.7 s, sys: 3 s, total: 15.7 s
Wall time: 17.5 s


In [20]:

cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(messages['cleaned']).toarray()
y = messages['label']

In [21]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression()

In [23]:
clf.fit(X_train, y_train)

LogisticRegression()

In [24]:
y_pred = clf.predict(X_test)
y_pred

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [25]:

	
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.9838565022421525
